<a href="https://colab.research.google.com/github/Raais/Blender-Colab/blob/main/Blender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
drive_path = 'file.blend' #@param {type: 'string'}
#@markdown ---
animation = False #@param {type: 'boolean'}
start_frame =  1#@param {type: 'integer'}
end_frame =  250#@param {type: 'integer'}
#@markdown ---
download_type = 'direct' #@param ['direct', 'google_drive', 'gdrive_direct'] {allow-input: false}
output_name = 'blender-##' #@param {type: 'string'}
zip_files = True #@param {type: 'boolean'}
drive_output_path = 'blender/output' #@param {type: 'string'}
#@markdown ---
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}

In [ ]:
import os
import shutil
from google.colab import files, drive

drive.mount('/drive')

In [ ]:
!mkdir render

In [ ]:
!cp -r '/drive/MyDrive/{drive_path}' 'render/'

In [ ]:
#find latest version 'linux-x64.tar.xz' at "https://ftp.nluug.nl/pub/graphics/blender/release/"
blender_version = '2.93.0'
blender_url = "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.93/blender-2.93.0-linux-x64.tar.xz"
base_url = os.path.basename(blender_url)

!mkdir $blender_version
!wget -nc $blender_url
!tar -xkf $base_url -C ./$blender_version --strip-components=1

--2021-06-19 15:01:58--  https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.93/blender-2.93.0-linux-x64.tar.xz
Resolving ftp.nluug.nl (ftp.nluug.nl)... 145.220.21.40, 2001:67c:6ec:221:145:220:21:40
Connecting to ftp.nluug.nl (ftp.nluug.nl)|145.220.21.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155802244 (149M) [application/x-tar]
Saving to: ‘blender-2.93.0-linux-x64.tar.xz’

blender-2.93.0-linu 100%[===================>] 148.58M  21.6MB/s    in 9.4s    

2021-06-19 15:02:09 (15.7 MB/s) - ‘blender-2.93.0-linux-x64.tar.xz’ saved [155802244/155802244]



In [ ]:
#Verify installation
!sudo ./$blender_version/blender -v

Blender 2.93.0
	build date: 2021-06-02
	build time: 11:29:24
	build commit date: 2021-06-02
	build commit time: 11:21
	build hash: 84da05a8b806
	build platform: Linux
	build type: release
	build c flags:  -Wall -Wcast-align -Werror=implicit-function-declaration -Werror=return-type -Werror=vla -Wstrict-prototypes -Wmissing-prototypes -Wno-char-subscripts -Wno-unknown-pragmas -Wpointer-arith -Wunused-parameter -Wwrite-strings -Wlogical-op -Wundef -Winit-self -Wmissing-include-dirs -Wno-div-by-zero -Wtype-limits -Wformat-signedness -Wrestrict -Wnonnull -Wabsolute-value -Wuninitialized -Wredundant-decls -Wshadow -Wno-error=unused-but-set-variable -Wimplicit-fallthrough=5  -fuse-ld=gold -std=gnu11 -msse -pipe -fPIC -funsigned-char -fno-strict-aliasing -ffp-contract=off -msse2 -D_GLIBCXX_USE_CXX11_ABI=0  
	build c++ flags:  -Wuninitialized -Wredundant-decls -Wall -Wno-invalid-offsetof -Wno-sign-compare -Wlogical-op -Winit-self -Wmissing-include-dirs -Wno-div-by-zero -Wtype-limits -Werror=ret

In [ ]:
# Enable GPU rendering (or add custom properties here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

In [ ]:
!mkdir output

if not drive_output_path.endswith('/'):
    drive_output_path += '/'

if download_type != 'gdrive_direct':
    output_path = 'output/' + output_name
else:
    output_path = '/drive/My Drive/' + drive_output_path + output_name

if animation:
    if start_frame == end_frame:
        !sudo ./$blender_version/blender -b 'render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -a
    else:
        !sudo ./$blender_version/blender -b 'render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -s $start_frame -e $end_frame -a
else:
    !sudo ./$blender_version/blender -b 'render/{blend_file_path}' -P setgpu.py -E CYCLES -o '{output_path}' -noaudio -f $start_frame

Simple export:

In [ ]:
!cp -r 'output/' '/drive/MyDrive/'

Other exports:

In [ ]:
path, dirs, files_folder = next(os.walk("output"))
output_folder_name = output_name.replace('#', '') + 'render'

if download_type == 'gdrive_direct':
    pass
elif len(files_folder) == 1:
    render_img = 'output/' + files_folder[0]
    if download_type == 'direct':
        files.download('output/' + files_folder[0])
    else:
        shutil.copy('/content/' + render_img, '/drive/My Drive/' + drive_output_path)
elif len(files_folder) > 1:
    if zip_files:
        shutil.make_archive(output_folder_name, 'zip', 'output')
    if download_type == 'direct':
        files.download(output_folder_name + '.zip')
    else:
        shutil.copy('/content/' + output_folder_name + ".zip", '/drive/My Drive/' + drive_output_path)
elif download_type == 'direct':
    for f in files_folder:
        files.download('output/{}'.format(f))
    # Drive, no zip
    else:
        for f in files_folder:
          shutil.copy("/content/output/" + f, '/drive/My Drive/' + drive_output_path + f)
else:
    raise SystemExit("No frames are rendered.")

## Disclaimer
The GPU used in Google Colab is specialized for data centres, neural network etc, not rendering 3D scenes. Because the computing power provided are free, the usage limits, idle timeouts and speed of the rendering may varies. [ColabPro](https://colab.research.google.com/signup) is available for those who wanted to have more powerful GPU and longer session for rendering. See the [FAQ](https://research.google.com/colaboratory/faq.html) for more info about this platform. In some cases, it might be faster to use a renderfarm such as [Sheepit](https://www.sheepit-renderfarm.com/) (free) and [ConciergeRender](https://www.conciergerender.com/) (paid w/ trial) which support parallel rendering.

## License
```
MIT License

Copyright (c) 2020 syn73

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```